# Brain Tumor Video Classification using PyTorch + W&B Tracking

## Import libraries

In [1]:
import os
import sys
import re
import gc
import platform
import random

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import einops

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split

import timm
import glob
import cv2
import pydicom as dicom

from rich import print as _pprint
from rich.progress import track

import albumentations as A
from albumentations.pytorch import ToTensorV2

import wandb

import warnings
warnings.simplefilter('ignore')

working_dir = '/home/denis/AI/python/clasificador de imagen/kaggle brain tumor/data'

In [106]:
#Código para saber cuantas imagenes tiene cada ejemplo (minimo y maximo)
working_dir2 = working_dir + '/rsna-miccai-brain-tumor-radiogenomic-classification' + '/train'
samples = os.listdir(working_dir2)
len_list = []
for s in samples:
    working_dir3 = working_dir2 + '/' + s + '/FLAIR'
    len_list.append(len(os.listdir(working_dir3)))

print(np.array(len_list).min(), np.array(len_list).max())


r = np.random.rand(2,5,3)
z = np.zeros((4,5,3))
s = np.concatenate((r,z), axis=0)
s.shape


frames_tr
h, w = frames_tr.shape 

z = np.zeros( (h,w,,max(100)-))
MAX_FRAMES

15 514


(6, 5, 3)

## Utility functions

In [17]:
def get_patient_id(patient_id):
    if patient_id < 10:
        return '0000'+str(patient_id)
    elif patient_id >= 10 and patient_id < 100:
        return '000'+str(patient_id)
    elif patient_id >= 100 and patient_id < 1000:
        return '00'+str(patient_id)
    else:
        return '0'+str(patient_id)

def get_path(row):
    patient_id = get_patient_id(row.BraTS21ID)
    return f'{working_dir}/rsna-miccai-brain-tumor-radiogenomic-classification/train/{patient_id}/FLAIR/'

def wandb_log(**kwargs):
    """
    Logs a key-value pair to W&B
    """
    for k, v in kwargs.items():
        wandb.log({k: v})
        
def cprint(string):
    """
    Utility function for beautiful colored printing.
    """
    _pprint(f"[black]{string}[/black]")

## Config Dictionary and W&B Integration

In [60]:
Config = dict(
    MAX_FRAMES = 300,
    EPOCHS = 5,
    LR = 2e-4,
    IMG_SIZE = (224, 224),
    FEATURE_EXTRACTOR = 'resnext50_32x4d',
    DR_RATE = 0.35,
    NUM_CLASSES = 1,
    RNN_HIDDEN_SIZE = 100,
    RNN_LAYERS = 1,
    TRAIN_BS = 4,
    VALID_BS = 4,
    NUM_WORKERS = 4,
    infra = "Kaggle",
    competition = 'rsna_miccai',
    _wandb_kernel = 'tanaym'
)

In [61]:
wandb.login()
run = wandb.init(
    project='kaggle_brain_tumor',
    config=Config,
    group='vision',
    job_type='train',
    anonymous='allow'
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


batch_train_loss,0.56779
batch_val_loss,0.68992
train_loss,0.69073
valid_loss,0.68715


batch_train_loss,▇▅▇▃▅▄█▃▃▄▃▂▄▁▇▅▃▆▂▅▄▄▄▃▅▁▄▃▃▃▁▄▄▄▆▃▁▄▂▃
batch_val_loss,▃▁▄▇█▄▄▃▅▆█▆▆▆▅▆▆██▇▇▇▆▆▆▇▅▇█▅▆▇▆▇▆▇▇▆▆▆
train_loss,▇█▁▂▂
valid_loss,▃▁▇█▂


In [62]:
class Augments:
    """
    Contains Train, Validation Augments
    """
    train_augments = A.Compose([
        ToTensorV2(p=1.0),
    ],p=1.)
    
    valid_augments = A.Compose([
        ToTensorV2(p=1.0),
    ], p=1.)

## Custom Dataset Class

In this custom Dataset, I am essentially reading "MAX_FRAMES" number of images from a patient's FLAIR folder and making list of those frames and converting it to torch tensor.

In [63]:
class RSNADataset(Dataset):
    def __init__(self, df, augments=None, is_test=False):
        self.df = df
        self.augments = augments
        self.is_test = is_test
        
    def __getitem__(self, idx):
        
        row = self.df.loc[idx]
        paths = self.getPaths(row)
        frames = []
        for path in paths:
            data_file = dicom.dcmread(path)
            img = data_file.pixel_array
            img = img.astype(np.uint8)
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            img = cv2.resize(img, Config['IMG_SIZE'])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            frames.append(img)

        frames_tr = np.stack(frames, axis=2)
        
        if self.augments:
            for frame in frames:
                frame = self.augments(image=frame)['image']
                frames_tr.append(frame)
            
        if self.is_test:
            return frames_tr
        else:
            label = torch.tensor(row['MGMT_value']).float()
            return frames_tr, label
        
    def __len__(self):
        return len(self.df)
    
    def getPaths(self, row):
        paths = glob.glob(row['path'] + '*.dcm')
        sortedPaths = self.sort(paths)
        maxWindowStart = len(sortedPaths) - Config['MAX_FRAMES']
        start = 0 # np.random.randint(1, maxWindowStart)
        paths = sortedPaths[start:Config['MAX_FRAMES']]
        
        return paths
        
    def sort(self, entry):
        # https://stackoverflow.com/a/2669120/7636462
        convert = lambda text: int(text) if text.isdigit() else text 
        alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    
        return sorted(entry, key = alphanum_key)

## Model Class with ResNext Backbone

In [64]:
class ResNextModel(nn.Module):
    def __init__(self):
        super(ResNextModel, self).__init__()
        self.backbone = timm.create_model(Config['FEATURE_EXTRACTOR'], pretrained=True, in_chans=1)
    def forward(self, x):
        return self.backbone(x)

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

class RSNAModel(nn.Module):
    def __init__(self, pretrained=True):
        super(RSNAModel, self).__init__()
        self.backbone = ResNextModel()
        num_features = self.backbone.backbone.fc.in_features
        
        self.backbone.backbone.fc = Identity()
        self.dropout= nn.Dropout(Config['DR_RATE'])
        self.rnn = nn.LSTM(num_features, Config['RNN_HIDDEN_SIZE'], Config['RNN_LAYERS'])
        self.fc1 = nn.Linear(Config['RNN_HIDDEN_SIZE'], Config['NUM_CLASSES'])
        
    def forward(self, x):
        b_z, fr, h, w = x.shape
        ii = 0
        in_pass = x[:, ii].unsqueeze_(1)
        y = self.backbone((in_pass))
        output, (hn, cn) = self.rnn(y.unsqueeze(1))
        for ii in range(1, fr):
            y = self.backbone((x[:, ii].unsqueeze_(1)))
            out, (hn, cn) = self.rnn(y.unsqueeze(1), (hn, cn))
        out = self.dropout(out[:, -1])
        out = self.fc1(out)
        return out

## Training and Validation Functions

In [65]:
def train_one_epoch(model, train_dataloader, optimizer, loss_fn, epoch, device, log_wandb=True, verbose=False):
    """
    Trains model for one epoch
    """
    model.train()
    running_loss = 0
    prog_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
    
    for batch, (frames, targets) in prog_bar:
        optimizer.zero_grad()
        
        frames = frames.to(device, torch.float)
        targets = targets.to(device, torch.float)
        
        # Re arrange the frames in the format our model wants to recieve
        frames = einops.rearrange(frames, 'b h w f -> b f h w')

        preds = model(frames).view(-1)
        loss = loss_fn(preds, targets)
        
        loss.backward()
        optimizer.step()
        
        loss_item = loss.item()
        running_loss += loss_item
        
        prog_bar.set_description(f"loss: {loss_item:.4f}")
        
        if log_wandb == True:
            wandb_log(
                batch_train_loss=loss_item
            )
        
        if verbose == True and batch % 20 == 0:
            print(f"Batch: {batch}, Loss: {loss_item}")
    
    avg_loss = running_loss / len(train_dataloader)
    
    return avg_loss

@torch.no_grad()
def valid_one_epoch(model, valid_dataloader, loss_fn, epoch, device, log_wandb=True, verbose=False):
    """
    Validates the model for one epoch
    """
    model.eval()
    running_loss = 0
    prog_bar = tqdm(enumerate(valid_dataloader), total=len(valid_dataloader))
    for batch, (frames, targets) in prog_bar:
        frames = frames.to(device, torch.float)
        targets = targets.to(device, torch.float)
        
        # Re arrange the frames in the format our model wants to recieve
        frames = einops.rearrange(frames, 'b h w f -> b f h w')
        preds = model(frames).view(-1)
        loss = loss_fn(preds, targets)
        
        loss_item = loss.item()
        running_loss += loss_item
        
        prog_bar.set_description(f"val_loss: {loss_item:.4f}")
        
        if log_wandb == True:
            wandb_log(
                batch_val_loss=loss_item
            )
        
        if verbose == True and batch % 10 == 0:
            print(f"Batch: {batch}, Loss: {loss_item}")
    
    avg_val_loss = running_loss / len(valid_dataloader)
    
    return avg_val_loss

In [66]:
if __name__ == "__main__":
    log_wandb = True
    if torch.cuda.is_available():
        print("Using GPU: {}\n".format(torch.cuda.get_device_name()))
        device = torch.device('cuda')
    else:
        print("\nGPU not found. Using CPU: {}\n".format(platform.processor()))
        device = torch.device('cpu')
    
    
    # Load training csv file
    df = pd.read_csv(working_dir+'/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
    df['path'] = df.apply(lambda row: get_path(row), axis=1)

    # Removing two patient ids from the dataframe since there are not FLAIR directories for these ids. 
    df = df.loc[df.BraTS21ID!=109]
    df = df.loc[df.BraTS21ID!=709]
    df = df.reset_index(drop=True)
    
    train_df, valid_df = train_test_split(df, test_size=0.1, stratify=df.MGMT_value.values)
    train_df = train_df.reset_index(drop=True)
    valid_df = valid_df.reset_index(drop=True)
    
    print(f'Size of Training Set: {len(train_df)}, Validation Set: {len(valid_df)}')
    
    model = RSNAModel()
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=Config['LR'])

    train_loss_fn = nn.BCEWithLogitsLoss()
    valid_loss_fn = nn.BCEWithLogitsLoss()
    
    print(f"\nUsing Backbone: {Config['FEATURE_EXTRACTOR']}")
    
    train_data = RSNADataset(train_df)
    valid_data = RSNADataset(valid_df)
    
    train_loader = DataLoader(
        train_data,
        batch_size=Config['TRAIN_BS'], 
        shuffle=True,
        num_workers=Config['NUM_WORKERS']
    )
    
    valid_loader = DataLoader(
        valid_data, 
        batch_size=Config['VALID_BS'], 
        shuffle=False,
        num_workers=Config['NUM_WORKERS']
    )
    
    current_loss = 1000
    for epoch in range(Config['EPOCHS']):
        print(f"\n{'--'*8} EPOCH: {epoch+1} {'--'*8}\n")
        
        train_loss = train_one_epoch(model, train_loader, optimizer, train_loss_fn, epoch=epoch, device=device, log_wandb=log_wandb)
        
        valid_loss = valid_one_epoch(model, valid_loader, valid_loss_fn, epoch=epoch, device=device, log_wandb=log_wandb)
        
        print(f"val_loss: {valid_loss:.4f}")
        
        if log_wandb == True:
            wandb_log(
                train_loss=train_loss,
                valid_loss=valid_loss
            )
        
        if valid_loss < current_loss:
            current_loss = valid_loss
            torch.save(model.state_dict(), f"model_{Config['FEATURE_EXTRACTOR']}.pt")

Using GPU: NVIDIA GeForce RTX 3060

Size of Training Set: 524, Validation Set: 59

Using Backbone: resnext50_32x4d

---------------- EPOCH: 1 ----------------



  0%|          | 0/131 [00:00<?, ?it/s]


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/denis/miniconda3/envs/deepsing/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/denis/miniconda3/envs/deepsing/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/home/denis/miniconda3/envs/deepsing/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py", line 84, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/denis/miniconda3/envs/deepsing/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py", line 84, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/home/denis/miniconda3/envs/deepsing/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py", line 64, in default_collate
    return default_collate(